In [2]:
# https://www.data.go.kr/data/15073855/openapi.do
import requests

url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc'
params ={'serviceKey' : 'fLoVrDwSn9qBTNUOApqBrjY+d99UJWe32K8T77qE77kvTXEXoHsx8KUde+p4vO+loILHU5VuVz4aXXUTuNT4Ig==' , 'returnType' : 'json', 'numOfRows' : '100', 'pageNo' : '1', 'sidoName' : '서울', 'ver' : '1.0' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\n<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">\n    <soapenv:Body>\n        <soapenv:Fault>\n            <faultcode>soapenv:Server</faultcode>\n            <faultstring>Policy Falsified</faultstring>\n            <faultactor>http://apis.data.go.kr/B552584/ArpltnInforInqireSvc?serviceKey=fLoVrDwSn9qBTNUOApqBrjY%2Bd99UJWe32K8T77qE77kvTXEXoHsx8KUde%2Bp4vO%2BloILHU5VuVz4aXXUTuNT4Ig%3D%3D&amp;returnType=json&amp;numOfRows=100&amp;pageNo=1&amp;sidoName=%EC%84%9C%EC%9A%B8&amp;ver=1.0</faultactor>\n            <detail>\n                <l7:policyResult\n                    status="Service Not Found.  The request may have been sent to an invalid URL, or intended for an unsupported operation." xmlns:l7="http://www.layer7tech.com/ws/policy/fault"/>\n            </detail>\n        </soapenv:Fault>\n    </soapenv:Body>\n</soapenv:Envelope>\n'


In [16]:
import json
import requests

def get_air_quality_data(sido, service_key, num_of_rows=100, page_no=1, version='1.0', return_type='json'):
    """
    해당 시도의 대기오염 정보를 조회하는 함수.

    Parameters:
    - sido (str): 시도 이름 (예: '서울', '경기')
    - service_key (str): API 인증키
    - num_of_rows (int): 한 페이지에 조회할 데이터 수 (기본값: 100)
    - page_no (int): 조회할 페이지 번호 (기본값: 1)
    - version (str): API 버전 (기본값: '1.0')
    - return_type (str): 응답 데이터 타입 (json 또는 xml, 기본값: 'json')

    Returns:
    - dict: API 호출 결과가 성공적일 경우 데이터 반환
    - None: 호출에 실패하거나 데이터를 얻지 못한 경우
    """
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'  # 엔드포인트 확인
    params = {
        'serviceKey': service_key,
        'returnType': return_type,
        'numOfRows': str(num_of_rows),
        'pageNo': str(page_no),
        'sidoName': sido,  # "서울"처럼 시/도 단위로 조회
        'ver': version
    }
    
    response = requests.get(url, params=params)
    
    # HTTP 상태 코드 확인
    if response.status_code != 200:
        print(f"HTTP Error: {response.status_code}")
        print(response.text)
        return None
    
    content = response.content.decode('utf-8')
    
    # 응답 내용 확인
    if not content.strip():  # 빈 응답 처리
        print("Received empty response from the server.")
        return None
    
    try:
        data = json.loads(content)
        return data
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        print("Response content:", content)
        return None

# 함수 사용 예시
service_key = 'fLoVrDwSn9qBTNUOApqBrjY+d99UJWe32K8T77qE77kvTXEXoHsx8KUde+p4vO+loILHU5VuVz4aXXUTuNT4Ig=='
sido_name = '서울'  # '서울'로 시/도 단위로 조회

data = get_air_quality_data(sido=sido_name, service_key=service_key)

if data and data['response']['body']['totalCount'] > 0:
    for item in data['response']['body']['items']:
        print(item)
else:
    print("No data found for the specified city.")



{'so2Grade': '1', 'coFlag': None, 'khaiValue': '92', 'so2Value': '0.003', 'coValue': '0.3', 'pm25Flag': None, 'pm10Flag': None, 'o3Grade': '2', 'pm10Value': '23', 'khaiGrade': '2', 'pm25Value': '18', 'sidoName': '서울', 'no2Flag': None, 'no2Grade': '1', 'o3Flag': None, 'pm25Grade': '1', 'so2Flag': None, 'dataTime': '2024-09-26 15:00', 'coGrade': '1', 'no2Value': '0.009', 'stationName': '중구', 'pm10Grade': '1', 'o3Value': '0.081'}
{'so2Grade': '1', 'coFlag': None, 'khaiValue': '93', 'so2Value': '0.003', 'coValue': '0.5', 'pm25Flag': None, 'pm10Flag': None, 'o3Grade': '2', 'pm10Value': '41', 'khaiGrade': '2', 'pm25Value': '18', 'sidoName': '서울', 'no2Flag': None, 'no2Grade': '1', 'o3Flag': None, 'pm25Grade': '2', 'so2Flag': None, 'dataTime': '2024-09-26 15:00', 'coGrade': '1', 'no2Value': '0.026', 'stationName': '한강대로', 'pm10Grade': '2', 'o3Value': '0.082'}
{'so2Grade': '1', 'coFlag': None, 'khaiValue': '94', 'so2Value': '0.003', 'coValue': '0.4', 'pm25Flag': None, 'pm10Flag': None, 'o3Grade

In [17]:
def parse_air_quality_data(data):
    items = data['response']['body']['items']
    air_quality_info = []
    for item in items:
        info = {
            '측정소명': item.get('stationName'),
            '날짜': item.get('dataTime'),
            'pm10농도': item.get('pm10Value'),
            'pm25농도': item.get('pm25Value'),
            'so2농도': item.get('so2Value'),
            'co농도': item.get('coValue'),
            'o3농도': item.get('o3Value'),
            'no2농도': item.get('no2Value'),
            '통합대기환경수치': item.get('khaiValue'),
            '통합대기환경지수': item.get('khaiGrade'),
            'pm10등급': item.get('pm10Grade'),
            'pm25등급': item.get('pm25Grade')
        }
        air_quality_info.append(info)
    return air_quality_info
 
air_quality_info = parse_air_quality_data(data)
print(air_quality_info)

[{'측정소명': '중구', '날짜': '2024-09-26 15:00', 'pm10농도': '23', 'pm25농도': '18', 'so2농도': '0.003', 'co농도': '0.3', 'o3농도': '0.081', 'no2농도': '0.009', '통합대기환경수치': '92', '통합대기환경지수': '2', 'pm10등급': '1', 'pm25등급': '1'}, {'측정소명': '한강대로', '날짜': '2024-09-26 15:00', 'pm10농도': '41', 'pm25농도': '18', 'so2농도': '0.003', 'co농도': '0.5', 'o3농도': '0.082', 'no2농도': '0.026', '통합대기환경수치': '93', '통합대기환경지수': '2', 'pm10등급': '2', 'pm25등급': '2'}, {'측정소명': '종로구', '날짜': '2024-09-26 15:00', 'pm10농도': '24', 'pm25농도': '16', 'so2농도': '0.003', 'co농도': '0.4', 'o3농도': '0.083', 'no2농도': '0.016', '통합대기환경수치': '94', '통합대기환경지수': '2', 'pm10등급': '1', 'pm25등급': '1'}, {'측정소명': '청계천로', '날짜': '2024-09-26 15:00', 'pm10농도': '30', 'pm25농도': '19', 'so2농도': '0.002', 'co농도': '0.5', 'o3농도': '0.068', 'no2농도': '0.023', '통합대기환경수치': '-', '통합대기환경지수': None, 'pm10등급': None, 'pm25등급': None}, {'측정소명': '종로', '날짜': '2024-09-26 15:00', 'pm10농도': '32', 'pm25농도': '16', 'so2농도': '0.003', 'co농도': '0.5', 'o3농도': '0.067', 'no2농도': '0.024', '통합대기환경수치': '81', '통합대기

In [18]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
 
documents = [Document(page_content=", ".join([f"{key}: {str(info[key])}" for key in ['측정소명', '날짜', 'pm10농도', 'pm25농도', '통합대기환경수치']])) for info in air_quality_info]
print(documents)

[Document(metadata={}, page_content='측정소명: 중구, 날짜: 2024-09-26 15:00, pm10농도: 23, pm25농도: 18, 통합대기환경수치: 92'), Document(metadata={}, page_content='측정소명: 한강대로, 날짜: 2024-09-26 15:00, pm10농도: 41, pm25농도: 18, 통합대기환경수치: 93'), Document(metadata={}, page_content='측정소명: 종로구, 날짜: 2024-09-26 15:00, pm10농도: 24, pm25농도: 16, 통합대기환경수치: 94'), Document(metadata={}, page_content='측정소명: 청계천로, 날짜: 2024-09-26 15:00, pm10농도: 30, pm25농도: 19, 통합대기환경수치: -'), Document(metadata={}, page_content='측정소명: 종로, 날짜: 2024-09-26 15:00, pm10농도: 32, pm25농도: 16, 통합대기환경수치: 81'), Document(metadata={}, page_content='측정소명: 용산구, 날짜: 2024-09-26 15:00, pm10농도: 28, pm25농도: 18, 통합대기환경수치: 85'), Document(metadata={}, page_content='측정소명: 광진구, 날짜: 2024-09-26 15:00, pm10농도: 27, pm25농도: 20, 통합대기환경수치: 115'), Document(metadata={}, page_content='측정소명: 성동구, 날짜: 2024-09-26 15:00, pm10농도: 30, pm25농도: 17, 통합대기환경수치: 93'), Document(metadata={}, page_content='측정소명: 강변북로, 날짜: 2024-09-26 15:00, pm10농도: 33, pm25농도: -, 통합대기환경수치: -'), Document(metadata={

In [20]:
import json
import requests
sido = "서울"
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
params ={'serviceKey' : 'fLoVrDwSn9qBTNUOApqBrjY+d99UJWe32K8T77qE77kvTXEXoHsx8KUde+p4vO+loILHU5VuVz4aXXUTuNT4Ig==', 'returnType' : 'json', 'numOfRows' : '100', 'pageNo' : '1', 'sidoName' : sido, 'ver' : '1.0' }    
response = requests.get(url, params=params)
content = response.content.decode('utf-8')
data = json.loads(content)
print(data)

{'response': {'body': {'totalCount': 40, 'items': [{'so2Grade': '1', 'coFlag': None, 'khaiValue': '91', 'so2Value': '0.003', 'coValue': '0.5', 'pm25Flag': None, 'pm10Flag': None, 'o3Grade': '2', 'pm10Value': '27', 'khaiGrade': '2', 'pm25Value': '16', 'sidoName': '서울', 'no2Flag': None, 'no2Grade': '1', 'o3Flag': None, 'pm25Grade': '2', 'so2Flag': None, 'dataTime': '2024-09-26 16:00', 'coGrade': '1', 'no2Value': '0.022', 'stationName': '한강대로', 'pm10Grade': '1', 'o3Value': '0.079'}, {'so2Grade': '1', 'coFlag': None, 'khaiValue': '88', 'so2Value': '0.004', 'coValue': '0.3', 'pm25Flag': None, 'pm10Flag': None, 'o3Grade': '2', 'pm10Value': '24', 'khaiGrade': '2', 'pm25Value': '13', 'sidoName': '서울', 'no2Flag': None, 'no2Grade': '1', 'o3Flag': None, 'pm25Grade': '1', 'so2Flag': None, 'dataTime': '2024-09-26 16:00', 'coGrade': '1', 'no2Value': '0.015', 'stationName': '종로구', 'pm10Grade': '1', 'o3Value': '0.075'}, {'so2Grade': '1', 'coFlag': None, 'khaiValue': '-', 'so2Value': '0.002', 'coValue'

In [21]:
# '용산구'에 해당하는 데이터만 출력
for item in data['response']['body']['items']:
    if '용산' in item['stationName']:
        print(f"측정소: {item['stationName']}")
        print(f"측정 시간: {item['dataTime']}")
        print(f"미세먼지(PM10) 농도: {item['pm10Value']}")
        print(f"초미세먼지(PM2.5) 농도: {item['pm25Value']}")
        print(f"오존(O3) 농도: {item['o3Value']}")
        print(f"통합대기환경지수(KHAI) 등급: {item['khaiGrade']}")
        print("================================")


측정소: 용산구
측정 시간: 2024-09-26 16:00
미세먼지(PM10) 농도: 28
초미세먼지(PM2.5) 농도: 17
오존(O3) 농도: 0.064
통합대기환경지수(KHAI) 등급: 2
